# 🔍 ML Pipeline Data Investigation

## Phân tích tại sao pipeline loại bỏ quá nhiều dữ liệu

Dựa trên log, pipeline đang loại bỏ rất nhiều dữ liệu:
- **Bắt đầu**: 81,052 records
- **Sau validation price**: 8 records (loại bỏ 81,044 records!)
- **Cuối cùng**: 6 records

### Mục tiêu điều tra:
1. **Price range validation**: Tại sao 81,044 records bị loại bỏ do "invalid price"?
2. **Data types**: Kiểm tra kiểu dữ liệu của các cột quan trọng
3. **Value distributions**: Phân tích phân phối giá trị price và area
4. **Missing values**: Tìm hiểu tỷ lệ missing values
5. **Validation logic**: Xem xét logic validation có phù hợp không

---

## 1. Import Required Libraries

In [1]:
import sys
import os
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Add project paths
sys.path.append('/home/fer/data/real_estate_project/data_processing')
sys.path.append('/home/fer/data/real_estate_project/data_processing/common')
sys.path.append('/home/fer/data/real_estate_project/data_processing/ml')

# Import PySpark
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, sum as spark_sum, avg, min as spark_min, max as spark_max,
    stddev, isnan, isnull, when, lit, regexp_replace
)
from pyspark.sql.types import DoubleType, StringType

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [2]:
# Initialize Spark session
spark = (
    SparkSession.builder.appName("DataPipelineInvestigation")
    .config("spark.ui.port", "4050")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "4g")
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000")
    .getOrCreate()
)


print(f"✅ Spark session initialized: {spark.version}")
spark.sparkContext.setLogLevel("WARN")  # Reduce log noise

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/09 10:23:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark session initialized: 3.4.1


## 2. Load Gold Data

In [3]:
# Set up parameters (same as in pipeline)
property_type = "house"
date = "2024-06-07"  # Test date
lookback_days = 30

# Calculate date range
end_date = datetime.strptime(date, "%Y-%m-%d")
start_date = end_date - timedelta(days=lookback_days - 1)

print(f"📅 Loading data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"🏠 Property type: {property_type}")

# Try to load one day of data first to understand the structure
sample_date = "2024-06-07"
date_formatted = sample_date.replace("-", "")
gold_path = f"hdfs://namenode:9000/data/realestate/processed/gold/unified/{property_type}/{sample_date.replace('-', '/')}/*.parquet"

print(f"🔍 Sample path: {gold_path}")

try:
    # Load sample data
    sample_df = spark.read.parquet(gold_path)
    print(f"✅ Successfully loaded sample data from {sample_date}")
    print(f"📊 Sample records: {sample_df.count():,}")
    print(f"📊 Sample columns: {len(sample_df.columns)}")
except Exception as e:
    print(f"❌ Failed to load sample data: {str(e)}")

    # Try different path formats
    alternative_paths = [
        f"/data/realestate/processed/gold/unified/{property_type}/unified_*.parquet",
        f"/data/realestate/processed/gold/{property_type}/{sample_date.replace('-', '/')}/*.parquet",
        f"/data/realestate/processed/gold/{property_type}/unified_{property_type}_{date_formatted}.parquet"
    ]

    for alt_path in alternative_paths:
        try:
            sample_df = spark.read.parquet(alt_path)
            print(f"✅ Found data at alternative path: {alt_path}")
            print(f"📊 Records: {sample_df.count():,}")
            gold_path = alt_path
            break
        except:
            continue
    else:
        raise Exception("Could not find gold data at any expected path")

📅 Loading data from 2024-05-09 to 2024-06-07
🏠 Property type: house
🔍 Sample path: hdfs://namenode:9000/data/realestate/processed/gold/unified/house/2024/06/07/*.parquet
❌ Failed to load sample data: [PATH_NOT_FOUND] Path does not exist: hdfs://namenode:9000/data/realestate/processed/gold/unified/house/2024/06/07/*.parquet.


Exception: Could not find gold data at any expected path

## 3. Inspect Initial Data

In [ ]:
# Display schema
print("📋 SCHEMA INFORMATION:")
print("=" * 50)
for field in sample_df.schema.fields:
    print(f"  - {field.name}: {field.dataType.typeName()}")

print(f"\n📊 TOTAL COLUMNS: {len(sample_df.columns)}")
print(f"📊 TOTAL RECORDS: {sample_df.count():,}")

In [ ]:
# Show sample data for critical columns
critical_columns = ['id', 'price', 'area', 'latitude', 'longitude', 'district', 'ward']
available_critical = [col for col in critical_columns if col in sample_df.columns]

print("\n🔍 SAMPLE DATA (First 10 records):")
print("=" * 80)
sample_data = sample_df.select(*available_critical).limit(10)
sample_data.show(truncate=False)

# Show data types for these columns
print("\n📊 CRITICAL COLUMNS DATA TYPES:")
for col_name in available_critical:
    col_type = dict(sample_df.dtypes)[col_name]
    print(f"  - {col_name}: {col_type}")

## 4. Validate Critical Columns

In [ ]:
# Check null values and data quality
print("🔍 NULL VALUE ANALYSIS:")
print("=" * 50)

total_records = sample_df.count()
print(f"Total records: {total_records:,}\n")

for col_name in available_critical:
    null_count = sample_df.filter(col(col_name).isNull()).count()
    null_pct = (null_count / total_records) * 100 if total_records > 0 else 0

    print(f"{col_name}:")
    print(f"  - Null values: {null_count:,} ({null_pct:.2f}%)")

    # For string columns, also check empty strings
    col_type = dict(sample_df.dtypes)[col_name]
    if col_type == 'string':
        empty_count = sample_df.filter((col(col_name) == '') | (col(col_name) == 'null') | (col(col_name) == 'NULL')).count()
        empty_pct = (empty_count / total_records) * 100 if total_records > 0 else 0
        print(f"  - Empty/null strings: {empty_count:,} ({empty_pct:.2f}%)")

    print()

## 5. Investigate Invalid Price Records

In [ ]:
# Analyze price column in detail
if 'price' in sample_df.columns:
    print("💰 PRICE ANALYSIS:")
    print("=" * 50)

    # Get price column type
    price_type = dict(sample_df.dtypes)['price']
    print(f"Price column data type: {price_type}")

    # Show sample price values
    print("\n🔍 Sample price values:")
    sample_prices = sample_df.select('price').limit(20).collect()
    for i, row in enumerate(sample_prices):
        print(f"  {i+1}: {row['price']} (type: {type(row['price'])})")

    # Get basic statistics
    print("\n📊 Price statistics:")
    price_stats = sample_df.select(
        spark_min('price').alias('min_price'),
        spark_max('price').alias('max_price'),
        avg('price').alias('avg_price'),
        count('price').alias('count_price')
    ).collect()[0]

    print(f"  - Min: {price_stats['min_price']}")
    print(f"  - Max: {price_stats['max_price']}")
    print(f"  - Average: {price_stats['avg_price']:,.2f}" if price_stats['avg_price'] else "N/A")
    print(f"  - Non-null count: {price_stats['count_price']:,}")

    # Check for zero and negative values
    zero_count = sample_df.filter(col('price') == 0).count()
    negative_count = sample_df.filter(col('price') < 0).count()
    print(f"\n⚠️ Price issues:")
    print(f"  - Zero values: {zero_count:,}")
    print(f"  - Negative values: {negative_count:,}")

    # Test the actual pipeline validation logic
    print("\n🧪 TESTING PIPELINE VALIDATION LOGIC:")
    print("Pipeline filters: price > 100,000 AND price < 100,000,000,000")

    valid_price_count = sample_df.filter(
        (col('price') > 100000) &
        (col('price') < 100000000000) &
        (col('price').isNotNull())
    ).count()

    invalid_price_count = total_records - valid_price_count

    print(f"  - Records passing validation: {valid_price_count:,}")
    print(f"  - Records failing validation: {invalid_price_count:,}")
    print(f"  - Failure rate: {(invalid_price_count/total_records)*100:.2f}%")

else:
    print("❌ Price column not found in data!")

In [ ]:
# Analyze price ranges in detail
if 'price' in sample_df.columns:
    print("\n📈 PRICE RANGE ANALYSIS:")
    print("=" * 50)

    # Count records in different price ranges
    price_ranges = [
        ("< 100K", col('price') < 100000),
        ("100K - 1M", (col('price') >= 100000) & (col('price') < 1000000)),
        ("1M - 10M", (col('price') >= 1000000) & (col('price') < 10000000)),
        ("10M - 100M", (col('price') >= 10000000) & (col('price') < 100000000)),
        ("100M - 1B", (col('price') >= 100000000) & (col('price') < 1000000000)),
        ("> 1B", col('price') >= 1000000000),
        ("Null/Invalid", col('price').isNull())
    ]

    total_non_null = sample_df.filter(col('price').isNotNull()).count()

    for range_name, condition in price_ranges:
        count = sample_df.filter(condition).count()
        pct = (count / total_records) * 100 if total_records > 0 else 0
        print(f"  {range_name}: {count:,} records ({pct:.2f}%)")

    # Show examples of records being filtered out
    print("\n🔍 EXAMPLES OF FILTERED OUT RECORDS:")
    filtered_out = sample_df.filter(
        (col('price') <= 100000) |
        (col('price') >= 100000000000) |
        col('price').isNull()
    ).select('id', 'price', 'area', 'district').limit(10)

    if filtered_out.count() > 0:
        filtered_out.show(truncate=False)
    else:
        print("  No records would be filtered out with current validation logic")

## 6. Investigate Invalid Area Records

In [ ]:
# Analyze area column in detail
if 'area' in sample_df.columns:
    print("📐 AREA ANALYSIS:")
    print("=" * 50)

    # Get area column type
    area_type = dict(sample_df.dtypes)['area']
    print(f"Area column data type: {area_type}")

    # Show sample area values
    print("\n🔍 Sample area values:")
    sample_areas = sample_df.select('area').limit(20).collect()
    for i, row in enumerate(sample_areas):
        print(f"  {i+1}: {row['area']} (type: {type(row['area'])})")

    # Get basic statistics
    print("\n📊 Area statistics:")
    area_stats = sample_df.select(
        spark_min('area').alias('min_area'),
        spark_max('area').alias('max_area'),
        avg('area').alias('avg_area'),
        count('area').alias('count_area')
    ).collect()[0]

    print(f"  - Min: {area_stats['min_area']}")
    print(f"  - Max: {area_stats['max_area']}")
    print(f"  - Average: {area_stats['avg_area']:,.2f}" if area_stats['avg_area'] else "N/A")
    print(f"  - Non-null count: {area_stats['count_area']:,}")

    # Check for zero and negative values
    zero_count = sample_df.filter(col('area') == 0).count()
    negative_count = sample_df.filter(col('area') < 0).count()
    print(f"\n⚠️ Area issues:")
    print(f"  - Zero values: {zero_count:,}")
    print(f"  - Negative values: {negative_count:,}")

    # Test pipeline validation logic for area
    print("\n🧪 TESTING AREA VALIDATION LOGIC:")
    print("Pipeline filters: area > 5 AND area < 50,000")

    valid_area_count = sample_df.filter(
        (col('area') > 5) &
        (col('area') < 50000) &
        (col('area').isNotNull())
    ).count()

    invalid_area_count = total_records - valid_area_count

    print(f"  - Records passing validation: {valid_area_count:,}")
    print(f"  - Records failing validation: {invalid_area_count:,}")
    print(f"  - Failure rate: {(invalid_area_count/total_records)*100:.2f}%")

else:
    print("❌ Area column not found in data!")

## 7. Analyze Missing Values

In [ ]:
# Comprehensive missing value analysis
print("📊 MISSING VALUES ANALYSIS (All Columns):")
print("=" * 60)

missing_analysis = []

for col_name in sample_df.columns:
    # Count different types of missing values
    null_count = sample_df.filter(col(col_name).isNull()).count()

    col_type = dict(sample_df.dtypes)[col_name]

    if col_type == 'string':
        # For string columns, also count empty strings and 'null' strings
        empty_count = sample_df.filter(
            (col(col_name) == '') |
            (col(col_name) == 'null') |
            (col(col_name) == 'NULL') |
            (col(col_name) == 'None')
        ).count()
        total_missing = null_count + empty_count
    else:
        # For numeric columns, only count nulls
        total_missing = null_count

    missing_pct = (total_missing / total_records) * 100 if total_records > 0 else 0

    missing_analysis.append({
        'column': col_name,
        'type': col_type,
        'null_count': null_count,
        'total_missing': total_missing,
        'missing_pct': missing_pct
    })

# Sort by missing percentage
missing_analysis.sort(key=lambda x: x['missing_pct'], reverse=True)

# Display results
print(f"{'Column':<20} {'Type':<10} {'Missing':<10} {'Percentage':<12}")
print("-" * 60)

for item in missing_analysis:
    print(f"{item['column']:<20} {item['type']:<10} {item['total_missing']:<10,} {item['missing_pct']:<12.2f}%")

# Highlight columns with high missing rates
print("\n⚠️ COLUMNS WITH >30% MISSING VALUES:")
high_missing = [item for item in missing_analysis if item['missing_pct'] > 30]
if high_missing:
    for item in high_missing:
        print(f"  - {item['column']}: {item['missing_pct']:.1f}% missing")
else:
    print("  None")

## 8. Investigate Outliers

In [ ]:
# Outlier analysis using IQR method (same as pipeline)
print("📊 OUTLIER ANALYSIS (IQR Method):")
print("=" * 50)

numeric_columns = ['price', 'area']
if 'price_per_m2' in sample_df.columns:
    numeric_columns.append('price_per_m2')

for col_name in numeric_columns:
    if col_name in sample_df.columns:
        print(f"\n🔍 Analyzing outliers in {col_name}:")

        # Calculate quantiles (same as pipeline)
        quantiles = sample_df.approxQuantile(col_name, [0.25, 0.75], 0.01)

        if len(quantiles) == 2:
            q1, q3 = quantiles
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr

            print(f"  - Q1: {q1:,.2f}")
            print(f"  - Q3: {q3:,.2f}")
            print(f"  - IQR: {iqr:,.2f}")
            print(f"  - Lower bound: {lower_bound:,.2f}")
            print(f"  - Upper bound: {upper_bound:,.2f}")

            # Count outliers
            total_non_null = sample_df.filter(col(col_name).isNotNull()).count()

            outliers_count = sample_df.filter(
                (col(col_name) < lower_bound) |
                (col(col_name) > upper_bound)
            ).count()

            valid_count = sample_df.filter(
                (col(col_name) >= lower_bound) &
                (col(col_name) <= upper_bound) &
                col(col_name).isNotNull()
            ).count()

            outlier_pct = (outliers_count / total_non_null) * 100 if total_non_null > 0 else 0

            print(f"  - Total non-null records: {total_non_null:,}")
            print(f"  - Outliers: {outliers_count:,} ({outlier_pct:.2f}%)")
            print(f"  - Records after outlier removal: {valid_count:,}")

            # Show examples of outliers
            print(f"\n  📋 Examples of outliers:")
            outlier_examples = sample_df.filter(
                (col(col_name) < lower_bound) | (col(col_name) > upper_bound)
            ).select('id', col_name, 'area' if col_name != 'area' else 'price').limit(5)

            if outlier_examples.count() > 0:
                outlier_examples.show()
            else:
                print("    No outliers found")
        else:
            print(f"  ❌ Could not calculate quantiles for {col_name}")

## 9. Summary of Data Removal Reasons

In [ ]:
# Comprehensive analysis of data removal pipeline
print("📋 COMPREHENSIVE DATA REMOVAL ANALYSIS:")
print("=" * 70)

print(f"Starting records: {total_records:,}")
print("\n🔍 Step-by-step filtering simulation:")

# Step 1: Remove duplicates (simulate)
current_df = sample_df
print(f"\n1️⃣ After duplicate removal: {current_df.count():,} records")

# Step 2: Critical column validation
print("\n2️⃣ Critical column validation:")
critical_columns = ['price', 'area', 'latitude', 'longitude']

for col_name in critical_columns:
    if col_name in current_df.columns:
        before_count = current_df.count()

        col_type = dict(current_df.dtypes)[col_name]

        if col_type == 'string':
            # String validation
            current_df = current_df.filter(
                col(col_name).isNotNull() &
                (col(col_name) != '') &
                (col(col_name) != 'null') &
                (col(col_name) != 'NULL')
            )
        else:
            # Numeric validation
            if col_name == 'area':
                current_df = current_df.filter(col(col_name).isNotNull() & (col(col_name) >= 0))
            else:
                current_df = current_df.filter(col(col_name).isNotNull() & (col(col_name) > 0))

        after_count = current_df.count()
        removed = before_count - after_count
        print(f"   {col_name}: {before_count:,} → {after_count:,} (-{removed:,} records)")

# Step 3: Numeric range validation
print("\n3️⃣ Numeric range validation:")

# Price validation
if 'price' in current_df.columns:
    before_count = current_df.count()
    current_df = current_df.filter(
        (col('price') > 100000) &
        (col('price') < 100000000000) &
        col('price').isNotNull()
    )
    after_count = current_df.count()
    removed = before_count - after_count
    print(f"   Price range (100K-100B): {before_count:,} → {after_count:,} (-{removed:,} records)")

# Area validation
if 'area' in current_df.columns:
    before_count = current_df.count()
    current_df = current_df.filter(
        (col('area') > 5) &
        (col('area') < 50000) &
        col('area').isNotNull()
    )
    after_count = current_df.count()
    removed = before_count - after_count
    print(f"   Area range (5-50K sqm): {before_count:,} → {after_count:,} (-{removed:,} records)")

# Step 4: Missing value handling (columns with >30% missing)
print("\n4️⃣ Missing value column removal:")
high_missing_cols = [item['column'] for item in missing_analysis if item['missing_pct'] > 30]
if high_missing_cols:
    print(f"   Dropping columns: {high_missing_cols}")
    current_df = current_df.drop(*high_missing_cols)
    print(f"   Remaining columns: {len(current_df.columns)}")
else:
    print("   No columns to drop (none have >30% missing)")

# Step 5: Outlier removal
print("\n5️⃣ Outlier removal (IQR method):")
outlier_columns = ['price', 'area']
for col_name in outlier_columns:
    if col_name in current_df.columns:
        before_count = current_df.count()

        quantiles = current_df.approxQuantile(col_name, [0.25, 0.75], 0.01)
        if len(quantiles) == 2:
            q1, q3 = quantiles
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr

            current_df = current_df.filter(
                (col(col_name) >= lower_bound) &
                (col(col_name) <= upper_bound)
            )

            after_count = current_df.count()
            removed = before_count - after_count
            print(f"   {col_name} outliers: {before_count:,} → {after_count:,} (-{removed:,} records)")

final_count = current_df.count()
print(f"\n🎯 FINAL RESULT: {final_count:,} records")
print(f"📊 Total removed: {total_records - final_count:,} records ({((total_records - final_count)/total_records)*100:.1f}%)")

## 🎯 Kết luận và Khuyến nghị

### Vấn đề chính được phát hiện:

1. **Price Range Validation quá nghiêm ngặt**: 
   - Pipeline filter `price > 100,000 VND` có thể loại bỏ nhiều dữ liệu hợp lệ
   - Trong thị trường BDS Việt Nam, giá có thể thấp hơn 100k VND cho một số trường hợp đặc biệt

2. **Data Type Issues**:
   - Cần kiểm tra xem dữ liệu có đang ở dạng string và cần convert không
   - Price có thể đang stored ở dạng không mong đợi

3. **Missing Value Handling**:
   - Nhiều columns có tỷ lệ missing cao (>30%)
   - Cần xem xét strategy cho missing values

### Khuyến nghị:

1. **Điều chỉnh Price Validation**:
   - Giảm minimum price threshold từ 100,000 xuống 10,000 VND
   - Hoặc sử dụng percentile-based filtering thay vì hard thresholds

2. **Improve Data Type Handling**:
   - Kiểm tra và fix data type conversion logic
   - Add more robust string-to-numeric conversion

3. **Better Outlier Detection**:
   - Sử dụng domain-specific knowledge về thị trường BDS
   - Consider using percentile-based outlier detection

4. **Data Quality Monitoring**:
   - Add more detailed logging for each filtering step
   - Create data quality reports

In [ ]:
# Cleanup
spark.stop()
print("✅ Investigation completed. Spark session stopped.")